In [ ]:
import os
import re
import io
import sys
import glob
import json
import time
import shutil
import requests
import datetime
import urllib
import psycopg2
import numpy as np
import pandas as pd

import sqlalchemy as db
import sqlalchemy.orm
import sqlalchemy.ext.declarative
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 1

sys.path.append('../..')
from opencell import constants, file_utils
from opencell.cli import database_cli, fov_cli
from opencell.database import models, operations
from opencell.database import utils as db_utils

%aimport opencell.database.cytoscape_networks
from opencell.database import cytoscape_networks

from opencell.api import resources, payloads

In [ ]:
url = db_utils.url_from_credentials('../../db-credentials-test.json')
url = db_utils.url_from_credentials('../../db-credentials-dev.json')
url = db_utils.url_from_credentials('../../db-credentials-cap.json')

engine = db.create_engine(url)
session_factory = db.orm.sessionmaker(bind=engine)
Session = db.orm.scoped_session(session_factory)
url

models.Base.metadata.create_all(engine)

### Mass spec interactions

In [ ]:
lines = (
    Session.query(models.CellLine)
    .join(models.CrisprDesign)
    .filter(db.func.lower(models.CrisprDesign.target_name) == db.func.lower('MTOR'))
    .all()
)

len(lines)

In [ ]:
lines[0].id

In [ ]:
lines[0].get_best_pulldown()

In [ ]:
pulldown_id = 235  # CLTA
pulldown_id = 562  # POLR2F
pulldown_id = 236  # MTOR
pulldown_id = 234  # ATL3
pulldown_id = 128

target_pulldown = (
    Session.query(models.MassSpecPulldown)
    .filter(models.MassSpecPulldown.id == pulldown_id)
    .one()
)

# copied from PulldownResources
significant_hits = (
    Session.query(models.MassSpecHit)
    .options(
        db.orm.joinedload(models.MassSpecHit.protein_group, innerjoin=True)
            .joinedload(models.MassSpecProteinGroup.crispr_designs),
        db.orm.joinedload(models.MassSpecHit.protein_group, innerjoin=True)
            .joinedload(models.MassSpecProteinGroup.uniprot_metadata),
    )
    .filter(models.MassSpecHit.pulldown_id == pulldown_id)
    .filter(db.or_(
        models.MassSpecHit.is_minor_hit == True,  # noqa
        models.MassSpecHit.is_significant_hit == True  # noqa
    ))
    .all()
)

In [ ]:
len(target_pulldown.get_significant_hits())

In [ ]:
# hits that map to more than one crispr design
[
    (ind, len(hit.protein_group.crispr_designs)) 
    for ind, hit in enumerate(significant_hits) if len(hit.protein_group.crispr_designs) > 1
]

In [ ]:
[d.cell_lines for d in significant_hits[1].protein_group.crispr_designs]

In [ ]:
[d for d in significant_hits[9].protein_group.crispr_designs]

In [ ]:
target_pulldown

In [ ]:
# find the 'good' pulldowns in which one or more of the target's protein groups appear as a hit
interacting_pulldowns = target_pulldown.get_interacting_pulldowns()
(interacting_pulldowns)

In [ ]:
for pd in interacting_pulldowns:
    bait_hit = pd.get_bait_hit(only_one=True)
    if bait_hit:
        print(bait_hit.protein_group)

In [ ]:
# BIRC5 id
pg = target_pulldown.get_significant_hits()[1].protein_group
pg

In [ ]:
for pd in interacting_pulldowns:
    pg_ids = [hit.protein_group_id for hit in pd.get_significant_hits()]
    if pg.id in pg_ids:
        print(pd)

In [ ]:
for hit in target_pulldown.get_significant_hits():
    print(hit)

In [ ]:
target_pulldown.get_significant_hits()

### Interactor networks

In [ ]:
ensg_id = 'ENSG00000110917'
uniprot_metadata = (
    Session.query(models.UniprotMetadata)
    .filter(models.UniprotMetadata.ensg_id == ensg_id)
    .all()
)
len(uniprot_metadata)

In [ ]:
interacting_pulldowns = (
    Session.query(models.MassSpecPulldown)
    .join(models.MassSpecHit)
    .join(models.MassSpecProteinGroup)
    .join(models.ProteinGroupUniprotMetadataAssociation)
    .filter(db.or_(
        models.MassSpecPulldown.manual_display_flag == None,  # noqa
        models.MassSpecPulldown.manual_display_flag == True  # noqa
    ))
    .filter(
        models.ProteinGroupUniprotMetadataAssociation.uniprot_id.in_(
            [row.uniprot_id for row in uniprot_metadata]
        )
    )
    .filter(db.or_(
        models.MassSpecHit.is_minor_hit == True,  # noqa
        models.MassSpecHit.is_significant_hit == True  # noqa
    ))
    .all()
)
len(interacting_pulldowns)

In [ ]:
nodes, edges = cytoscape_networks.construct_network(
            interacting_pulldowns=interacting_pulldowns,
            uniprot_metadata=uniprot_metadata[0],
        )

In [ ]:
interacting_bait_hits = [
    interacting_pulldown.get_bait_hit(only_one=True)
    for interacting_pulldown in interacting_pulldowns
]